In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import inspect


# Loading Data Set

In [8]:
train_ds = keras.utils.image_dataset_from_directory(
    directory='baseline_training_set/train/',
    labels='inferred',
    label_mode='categorical',
    batch_size=32,
    image_size=(256, 256))

validation_ds = keras.utils.image_dataset_from_directory(
    directory='baseline_training_set/validation/',
    labels='inferred',
    label_mode='categorical',
    batch_size=32,
    image_size=(256, 256))
test_ds = keras.utils.image_dataset_from_directory(
    directory='baseline_training_set/test/',
    labels='inferred',
    label_mode='categorical',
    batch_size=32,
    image_size=(256, 256))
for element in test_ds.classes.as_numpy_iterator():
    print(element)

Found 22320 files belonging to 15 classes.
Found 2790 files belonging to 15 classes.
Found 2790 files belonging to 15 classes.


AttributeError: 'BatchDataset' object has no attribute 'classes'

In [5]:
base_model = ResNet50(weights='imagenet', # utilizing transfer learning with ImageNet as initial weights
                 input_shape=(256, 256, 3), # input dimensions (width: 255, height: 255: channels: 3)
                 include_top=False # not including the ImageNet classifier
                )


In [6]:
for layer in base_model.layers: # freeze the base model
    layer.trainable = False

In [23]:
model = keras.models.Sequential()
model.add(keras.layers.Lambda(lambda x: tf.image.resize(x, (224, 224))))
model.add(base_model)
model.add(keras.layers.GlobalAveragePooling2D())
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(256, activation='relu'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(10, activation='softmax'))

In [26]:
model.compile(loss='binary_crossentropy',
              optimizer=keras.optimizers.Adam(learning_rate=5*10**(-3)),
              metrics=['accuracy'])
print("Noice")

Noice


In [27]:
tf.keras.utils.plot_model(model, "resnet50.png")


ValueError: This model has not yet been built. Build the model first by calling `build()` or by calling the model on a batch of data.